# MutPanning

MutPanning is designed to detect rare cancer driver genes from aggregated whole-exome sequencing data. Most approaches detect cancer genes based on their mutational excess, i.e. they search for genes with an increased number of nonsynonymous mutations above the background mutation rate. MutPanning further accounts for the nucleotide context around mutations and searches for genes with an excess of mutations in unusual sequence contexts that deviate from the characteristic sequence context around passenger mutations.

In [5]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

## Introduction
MutPanning analyzes aggregated DNA sequencing data of tumor patients to identify genes that are likely to be functionally relevant, based on their abundance of nonsynonymous mutations or their increased number of mutations in unusual nucleotide contexts that deviate from the background mutational process. 
 
The name MutPanning is inspired by the words "mutation" and "panning". The goal of the MutPanning algorithm is to discover new tumor genes in aggregated sequencing data, i.e. to "pan" the few tumor-relevant driver mutations from the abundance of functionally neutral passenger mutations in the background. Previous approaches for cancer gene discovery were mostly based on mutational recurrence, i.e. they detected cancer genes based on their excess of nonsynonymous mutation above the local background mutation rate.  Further, they search for mutations that occur in functionally important genomic positions, as predicted by bioinformatical scores). These approaches are highly effective in tumor types, for which the average background mutation rate (i.e., the total mutational burden) is low or moderate.
 
The ability to detect driver genes can be increased by considering the nucleotide context around mutations in the statistical model. MutPanning utilizes the observation that most passenger mutations are surrounded by characteristic nucleotide sequence contexts, reflecting the background mutational process active in a given tumor. In contrast, driver mutations are localized towards functionally important positions, which are not necessarily surrounded by the same nucleotide contexts as passenger mutations. Hence, in addition to mutational excess, MutPanning searches for genes with an excess of mutations in unusual sequence contexts that deviate from the characteristic sequence context around passenger mutations. That way, MutPanning actively suppresses mutations in its test statistics that are likely to be passenger mutations based on their surrounding nucleotide contexts. Considering the nucleotide context is particularly useful in tumor types with high background mutation rates and high nucleotide context specificity (e.g., melanoma, bladder, endometrial, or colorectal cancer).
 
## Algorithm
Most passenger mutations occur in characteristic nucleotide contexts that reflect the mutational process active in a given tumor. MutPanning searches for mutations in “unusual” nucleotide contexts that deviate from this background mutational process. In these positions, passenger mutations are rare and mutations are thus a strong indicator of the shift of driver mutations towards functionally important positions.
 
The main steps of MutPanning are as follows (adopted from Dietlein et al.): 
(i) Model the mutation probability of each genomic position in the human exome depending on its surrounding nucleotide context and the regional background mutation rate. 
(ii) Given a gene with n nonsynonymous mutations, use a Monte Carlo simulation approach to simulate a large number of random “scenarios” in which n or more nonsynonymous mutations are randomly distributed along the same gene . 
(iii) Compare the number and positions of mutations in each random scenario with the observed mutations in gene . Based on these comparisons, derive a p-value for the gene. 
(iv) Combine this p-value with additional statistical components that account for insertions and deletions, the abundance of deleterious mutations, and mutational clustering.
 
The following figure (adopted from Dietlein et al.) illustrates how MutPanning works.

<p><img alt="" src="https://datasets.genepattern.org/images/MutPanning_Dietlen_Fig1.png" /></p>


## Input Data

MutPanning takes two input files, a mutation file which lists the positions of all somatic mutations in your cohort. 

### Mutation file
*The mutation file* file should follow the standard ([mutation annotation format.](http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide#MAF)). Several mutation callers (e.g. MuTect, MutSigCV, ABSOLUTE) report their somatic mutations in this file format. 

Each row corresponds to an individual mutation, which is annotated by the following colums.
 

**Hugo_Symbol** The nomenclature of the symbol in Hugo nomenclature (genenames.org).

**Chromosome** The chromosome, on which the mutation was found. Please use X and Y for the sex chromosomes and not 23 and 24.

**Start_Position** The start position of the mutation (Hg19).

**End_Position** The end position of the mutation (Hg19).

**Strand** The strand on which the mutation was detected (both 1/-1 or +/- nomenclature are fine).

**Variant_Classification** The functional class of the mutation (e.g. Silent, Missense_Mutations etc.). Please use the standard nomenclature used in MAF files. If you are unsure about the file format, please see the exemplary files below.

**Variant_Type** The type of the mutation, such as single base substitution (SNP), insersions (INS) or deletions (DEL). Please use the standard nomenclature used in MAF files. If you are unsure about the file format, please see the exemplary files below.

**Reference_Allele** The nucleotide expected in the Hg19 reference genome. Please note that if this mutation does not with the in the Hg19 genome, this mutation is ignored.

**Tumor_Seq_Allele1** The alternative nucleotide 1 found in the reads. 

**Tumor_Seq_Allele2** The alternative nucleotide 2 found in the reads. Typically, either of these nucleotides is the reference allele (allele A), whereas the other column contains the alternative read (allele B). Different callers handle differently how they assign these columns. MutPanning will first look in Tumor_Seq_Allele1 whether this equals the reference allele. If so, will take the change in the second Tumor_Seq_Allele column as alternative read.

**Tumor_Sample_Barcode** The name of the tumor sample. These identifies have to be unique and be exactly the same as in the sample annotation file (case-sensitive).

### Sample Annotation File

This file organizes samples into subcohorts, e.g. cancer types. Samples in each subcohort are analyzed together for mutational significance. Each row in this file corresponds to an individual sample, which is annotated by the following columns.

**Sample** The same sample identifier as used in the mutation annotation file (case-sensitive). Note that these sample identifiers must be unique. Avoid special characters.

**Cohort** The cohort name, in which the samples should be analyzed together for significance. This is typically the cancer type, but you can also group your samples by other criteria (e.g., subtypes of cancer types or combine different cancer types together).






## Parameters

The parameters governing how MutPanning finds mutations are as follows;

<table>
	<thead>
		<tr>
			<th>Name</th>
			<th>Description</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td>min samples per cluster*</td>
			<td>Minimum number of samples needed per nucleotide context cluster. Unless you are familiar with the MutPanning algorithm, we recommend running MutPanning with standard parameters (default value 3).</td>
		</tr>
		<tr>
			<td>min mutations per cluster*</td>
			<td>Minimum number of mutations needed per nucleotide context cluster. Unless you are familiar with the MutPanning algorithm, we recommend running MutPanning with standard parameters (default value 1000).</td>
		</tr>
		<tr>
			<td>min samples Bayesian model*</td>
			<td>Minimum number of samples needed to calibrate the Bayesian background model. Unless you are familiar with the MutPanning algorithm, we recommend running MutPanning with standard parameters (default value 100).</td>
		</tr>
		<tr>
			<td>min mutations Bayesian model*</td>
			<td>Minimum number of mutations needed to calibrate the Bayesian background model. Unless you are familiar with the MutPanning algorithm, we recommend running MutPanning with standard parameters (default value 5000).* - required</td>
		</tr>
	</tbody>
</table>


## Sample Data

For this notebook we will use a set of small ample files containing Melanoma Samples from The Cancer Genome Atlas (TCGA).


In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import os, sys

maf_path = 'http://storage.googleapis.com/mutpanning_hg19/MutationsSkin.maf'
sample_path = 'http://storage.googleapis.com/mutpanning_hg19/SamplesSkin.txt'

genepattern.GPUIOutput(files=[maf_path, sample_path])

UIOutput(files=['http://storage.googleapis.com/mutpanning_hg19/MutationsSkin.maf', 'http://storage.googleapis.…

## Running MutPanning on the sample data

Below you will find a run of MutPanning using the TCGA Melanoma dataset.  The results for Job #51724 are publicly visible and you may review them without re-running the analysis.  If you choose to run MutPanning yourself please note that it will take approximately 30 minutes to complete for this dataset.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<ol>
    <li>Drag the 'MutationSkin.maf' file (above) to the mutation file parameter input.</li>
    <li>Drag the 'SamplesSkin.txt' file (above) to the sample annotation file parameter input</li>
    <li>Leave the remaining parameters at their default values</li>
    <li>Click 'Run'</li>
</ol>
    <b>NOTE: Even with this small dataset, MutPanning will take 20-30 minutes to complete. </b>
</div>

In [6]:
mutpanning_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00380')
mutpanning_job_spec = mutpanning_task.make_job_spec()
mutpanning_job_spec.set_parameter("mutation.file", "")
mutpanning_job_spec.set_parameter("sample.annotation.file", "")
mutpanning_job_spec.set_parameter("min.samples.per.cluster", "3")
mutpanning_job_spec.set_parameter("min.mutations.per.cluster", "1000")
mutpanning_job_spec.set_parameter("min.samples.Bayesian.model", "100")
mutpanning_job_spec.set_parameter("min.mutations.Bayesian.model", "5000")
mutpanning_job_spec.set_parameter("job.memory", "16Gb")
mutpanning_job_spec.set_parameter("job.queue", "MutPanning")
mutpanning_job_spec.set_parameter("job.cpuCount", "1")
mutpanning_job_spec.set_parameter("job.walltime", "6:00:00")
genepattern.display(mutpanning_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00380')

## Interpreting the results


The output of MutPanning is a zipped folder, which contains a mutational significance report for each subcohort (e.g. cancer type). If the cohort size is too small to calibrate the Bayesian background model (typically for cancer types with a very low background mutation rate, cf. parameters), the folder contains two analysis reports, based on the Bayesian model and a uniform background model. 

Each significance report is a tab-delimited txt file, which lists genes in descending order according to their mutational significance. Each row corresponds to an individual gene, which is annotated by the following columns:

**Name** Gene name

**TargetSize** The nonsynonymous target size of the gene corrected for nucleotide context bias (sum of likelihood coefficients of nonsynonymous positions). 

**TargetSizeSyn** The synonymous target size of the gene corrected for nucleotide context bias (sum of likelihood coefficients of synonymous positions).

**Count** Number of nonsynonymous mutations 

**CountSyn** Number of synonymous mutations

**Significance** Mutational significance of the gene according to MutPanning. This p-value considers the excess of nonsynonymous mutations, the increased number of mutations in unusual nucleotide contexts, as well as insertions and deletions.

**FDR** The MutPanning p-value corrected for multiple hypothesis testing (q-value, Benjamini-Hochberg procedure).

To examine the results of our run, we will first download them from the remote GenePattern server and then load them into a local Pandas dataframe for easy manipulation.


<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
    <ol>
        <li>In the 'MutPanning Results' parameter below, select the Results.tar.gz file from the most recent run.</li>
        <li>Click the '<b>Run</b>' button.</li>
        <li>Select the desired results to look at from the dropdown.  (dropdown appears after the Run button is used)</li>
        <li>Adjust the max FDR as desired.</li>
    </ol></div>

In [4]:
# import os
import sys
import pandas as pd

def printAndFlush(str):
    print(str)
    sys.stdout.flush()
 

@genepattern.build_ui(parameters={
    "Mutpanning_Results": {
        "type": "file",
        "kinds": ["tar","tar.gz"]
    },
     "output_var": {
         "default": "mpdf",
         "hide": True
     }
})    
def DownloadMutPanningResultFile(Mutpanning_Results):
    remote_file_name = Mutpanning_Results.split("/")[-1]
    File_Name = remote_file_name
    print("Saving to notebook directory.")
    if os.path.isfile(File_Name):
        print("file already exists.  Replacing with new copy.")
        os.system('rm '+ File_Name)

    # extract job number and file name from url
    job_num = Mutpanning_Results.split("/")[-2]
    remote_file_name = Mutpanning_Results.split("/")[-1]
    
    # get the job based on the job number passed as the second argument
    job = gp.GPJob(genepattern.get_session(0), job_num)

    # fetch a specific file from the job
    remote_file = job.get_file(remote_file_name)
    
    response = remote_file.open()
    CHUNK = 16 * 1024
    with open(File_Name, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)
    
    # move to a subdir
    Dir_Name = File_Name[:File_Name.index('.')]
    print("Expanding results into new subfolder called "+ Dir_Name)
    
    os.system('mkdir ' + Dir_Name);
    os.system('mv ' + File_Name + ' ./'+Dir_Name); 
    os.system('cd '+Dir_Name+ '; tar xvf ' + File_Name);

    printAndFlush("Done downloading & expanding results.")
    
    #df = pd.read_csv('SignificanceSkin.txt',sep='\t',skiprows=(0),header=(0))
    #printAndFlush("Done. Mutpanning results saved and loaded as variable \'mpdf\'.  Head of dataframe is...")
    #display(df.head())
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(Dir_Name):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames if file.endswith(".txt")]

    def selectFile(result_file=listOfFiles, max_FDR=0.25):
        df = pd.read_csv(result_file,sep='\t',skiprows=(0),header=(0))
       
        display(df.loc[df['FDR'] < max_FDR])
    
    display(interactive(selectFile))
    return None


UIBuilder(function_import='nbtools.tool(id="DownloadMutPanningResultFile", origin="Notebook").function_or_meth…